In [1]:
import GLPE
#https://github.com/ekehoe32/orthrus
import sys
sys.path.append('/home/katrina/a/mankovic/ZOETIS/Fall2021/Orthrus/orthrus')
import orthrus
from orthrus import core
from orthrus.core import dataset
import numpy as np
# from NetworkDataAnalysis import graph_tools_construction as gt
from matplotlib import pyplot as plt
import pandas
# from orthrus.core.pipeline import *
from sklearn.preprocessing import FunctionTransformer
from orthrus.preprocessing.imputation import HalfMinimum
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from orthrus.core.helper import load_object

# GSE Test

In [2]:
# ds = pandas.read_csv('/data4/mankovic/GSE73072/calcom_splits/entrez_ids/datagse73072_5to1_25_32_train_limma.csv', index_col = 'SubjectID')
ds = pandas.read_csv('/data4/mankovic/GSE73072/calcom_splits/data/gse73072_5to1_25_32_train_limma.csv', index_col = 'SubjectID')

In [3]:
precomputed_clpe = GLPE.CLPE(centrality_measure = 'page_rank', 
                    network_type = 'precomputed',
                    feature_ids = list(ds.columns),
                    pathway_files = '/data4/mankovic/GSE73072/new_pw_edge_mtx/',
                    # pathway_files = '/data3/darpa/omics_databases/ensembl2pathway/pathways_edges/pathways/pw_edge_mtx/',
                    directed = False,
                    heat_kernel_param=2,
                    normalize_rows=False)

print(precomputed_clpe)

precomputed_clpe.fit(np.array(ds))

precomputed_clpe.pathway_transition_matrix_.shape, len(precomputed_clpe.pathway_names_)

CLPE(centrality_measure='page_rank',
     feature_ids=['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at',
                  '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at',
                  '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at',
                  '1729_at', '177_at', '1773_at', '179_at', '1861_at',
                  '200000_s_at', '200001_at', '200002_at', '200003_s_at',
                  '200004_at', '200005_at', '200006_at', '200007_at',
                  '200008_s_at', '200009_at', ...],
     heat_kernel_param=2.0, network_type='precomputed', normalize_rows=0.0,
     pathway_files='/data4/mankovic/GSE73072/new_pw_edge_mtx/')


((2261, 22277), 2261)

In [4]:
np.sum(precomputed_clpe.pathway_transition_matrix_)

1894.5213742354752

In [5]:
cor_clpe = GLPE.CLPE(centrality_measure = 'degree', 
                    network_type = 'correlation',
                    feature_ids = list(ds.columns),
                    pathway_files = '/data3/darpa/omics_databases/ensembl2pathway/pathways_edges/pathways/pw_edge_mtx/',
                    directed = False,
                    heat_kernel_param=2,
                    normalize_rows=False)

print(cor_clpe)

cor_clpe.fit(np.array(ds))

cor_clpe.pathway_transition_matrix_.shape, len(cor_clpe.pathway_names_)

CLPE(centrality_measure='degree',
     feature_ids=['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at',
                  '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at',
                  '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at',
                  '1729_at', '177_at', '1773_at', '179_at', '1861_at',
                  '200000_s_at', '200001_at', '200002_at', '200003_s_at',
                  '200004_at', '200005_at', '200006_at', '200007_at',
                  '200008_s_at', '200009_at', ...],
     heat_kernel_param=2.0, network_type='correlation', normalize_rows=0.0,
     pathway_files='/data3/darpa/omics_databases/ensembl2pathway/pathways_edges/pathways/pw_edge_mtx/')


((2261, 22277), 2261)

In [5]:
feats = pandas.read_csv('/data4/mankovic/GSE73072/features/5_datasets_with_limma_subject_id_training_25_32.csv', index_col = 0)
feat_ids = [str(f) for f in list(feats['EntrezID'])]
restricted_feat_ids = list(set(list(ds.columns)).intersection(list(feat_ids)))
res = precomputed_clpe.simple_transform(restricted_feat_ids, n_null_trials = 5)

null trial 0 done


In [2]:
#load data
Z40_dataset = dataset.load_dataset('/data4/mankovic/De-Identified_CZ/DeId_TPM_C1_Z40_Z34.ds')
Z40_dataset.metadata = Z40_dataset.metadata.query("Project == 'Z40' & Treatment == 'High' & Timepoint == 0.0")
sidx = list(Z40_dataset.metadata.index)  
Z40_dataset.data = Z40_dataset.data.loc[sidx]

#transform data (according to Kartikay)
transform = make_pipeline(HalfMinimum(missing_values=0), FunctionTransformer(np.log2))
Z40_dataset.normalize(transform, norm_name='HalfMinimum + Log2')
Z40_data = Z40_dataset.data





In [3]:
my_other_clpe = GLPE.CLPE(centrality_measure = 'degree', 
                    network_type = 'correlation',
                    feature_ids = list(Z40_data.columns),
                    pathway_files = '/data4/mankovic/De-Identified_CZ/deidentified_fcpw.csv',
                    directed = False,
                    heat_kernel_param=2,
                    normalize_rows=False)
my_other_clpe

CLPE(centrality_measure='degree',
     feature_ids=['1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '12',
                  '13', '14', '16', '17', '18', '19', '20', '21', '22', '23',
                  '24', '25', '26', '27', '30', '31', '32', '33', '34', ...],
     heat_kernel_param=2.0, network_type='correlation', normalize_rows=0.0,
     pathway_files='/data4/mankovic/De-Identified_CZ/deidentified_fcpw.csv')

In [4]:
len(list(Z40_data.columns))

28068

In [5]:
my_other_clpe.fit(np.array(Z40_data))

CLPE(centrality_measure='degree',
     feature_ids=['1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '12',
                  '13', '14', '16', '17', '18', '19', '20', '21', '22', '23',
                  '24', '25', '26', '27', '30', '31', '32', '33', '34', ...],
     heat_kernel_param=2.0, network_type='correlation', normalize_rows=0.0,
     pathway_files='/data4/mankovic/De-Identified_CZ/deidentified_fcpw.csv')

In [9]:
np.sum(my_other_clpe.pathway_transition_matrix_), my_other_clpe.pathway_transition_matrix.shape, np.array(Z40_data.shape)

(85521.4158050756, (1979, 28068), array([    8, 28068]))